# Wissensaggregator Mittelalter und frühe Neuzeit

## Strukturdaten einlesen/aktualisieren

Inhalt

- [User anlegen](#User-anlegen)
- [Typen anlegen](#Typen-anlegen)
- [SKOS-Typen einlesen](#SKOS-Typen-einlesen)
- [Quellen/Institutionen für externe IDs](#Quellen/Institutionen-für-externe-IDs)
- [Quellen/Referenzen](#Quellen/Referenzen)
- [Ämter](#Ämter)

Die Funktionen, welche Daten laden und umwandeln sind im Modul `WiagDataSetup` enthalten. Dieses verwendet seinerseits folgende Module:

- MySQL
- DataFrames
- JSON
- Infiltrator

Diese werden am besten vorab direkt in einem Julia-Terminal installiert.
``` julia
cd("C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl")
Pkg.activate(".")
Pkg.add("MySQL")
Pkg.add("DataFrames")
Pkg.add("JSON")
Pkg.add("Infiltrator")
```

Pfad zum Modul `WiagDataSetup`

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


Nur für die Entwicklung des Moduls relevant.

In [5]:
using Revise

Modul laden

In [6]:
using WiagDataSetup; Wds=WiagDataSetup

WiagDataSetup

Verbinde die Datenbank.

Im Verlauf der Einleseschritte kann die folgende Fehlermeldung erscheinen:
*Commands out of sync; you can't run this command now*.
In diesem Fall diesen Befehl nochmal absetzen.

In [8]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

Verzeichnis für Basisdaten, z.b. SKOS-Schemes, Länder

In [9]:
data_path="C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup"

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup"

In [10]:
wiag2_data_path="C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data"

## User anlegen

In [1]:
using CSV, DataFrames

In [20]:
user_file = joinpath(data_path, "csv", "user_wiag.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\user_wiag.csv"

In [37]:
df_user = CSV.read(user_file, DataFrame)

,id,note,user_id,nickname,givenname
,Int64,String?,Int64?,String7,String15?
1,1,"funktionaler User, z.B. für automatisierten Import",missing,wiag,missing
2,2,missing,11,bkroege,Bärbel
3,3,missing,12,cpopp,Christian
4,7,missing,13,ghertko,Georg


In [38]:
Wds.filltable!("user_wiag", df_user; clear_table=true)

┌ Info: Rows inserted: 4
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


4

## Typen anlegen

Alle Entitäten/items werden einem Typ zugeordnet, z.B. Diözese, Bischof, Domherr, Domherr aus GS. Der Typ bestimmt in welcher Tabelle Daten zu der Entität abgelegt werden. Es kann aber durchaus sein, dass auch unterschiedliche Typen in einer Tabelle versammelt sind, z.B. Bischöfe und Domherren in `person`.

In [9]:
using CSV, DataFrames

In [10]:
item_type_file = joinpath(data_path, "csv", "item_type.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\item_type.csv"

In [11]:
df_item_type = CSV.read(item_type_file, DataFrame)

,id,note,name
,Int64,String?,String15
1,1,Diözese des Alten Reiches,Diözese
2,2,missing,Kloster
3,3,missing,Domstift
4,4,Bischof oder Patriarch des Alten Reiches,Bischof
5,5,Domherr des Alten Reiches,Domherr
6,6,Domherr des Alten Reiches aus der Personendatenbank der Germania Sacra,Domherr GS
7,7,religiöser Orden,Orden
8,8,"Amt, Rolle",Amt


In [12]:
Wds.filltable!("item_type", df_item_type; clear_table=true)

┌ Info: Rows inserted: 8
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1184


8

## SKOS Typen einlesen
Für alle Entitäten, die einen SKOS-Typ besitzen, können SKOS-Beziehungen hinterlegt werden. Für diese Entitäten kann ein SKOS-Thesaurus aufgebaut werden und/oder es können Bezeichner *label* in verschiedenen Sprachen hinterlegt werden.

Quelle: händisch erstellt

In [9]:
using CSV, DataFrames

In [13]:
skos_scheme_file=joinpath(data_path, "csv", "skos_scheme.csv")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\csv\\skos_scheme.csv"

In [14]:
df_skos_scheme = CSV.read(skos_scheme_file, DataFrame)

,id,note,name
,Int64,String63?,String31
1,1,missing,Diözese
2,2,kirchliche Ämter,Amt
3,3,religiöser Orden,Orden
4,4,"z.B. Kloster, Domstift",Einrichtung
5,5,z.B. Bundesländer; genutzt für Übersetzungen,Verwaltungseinheit


In [15]:
Wds.filltable!("skos_scheme", df_skos_scheme, clear_table = true)

┌ Info: Rows inserted: 5
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1171


5

## Quellen/Institutionen für externe IDs

Quelle: Tabelle `wiag.external_url_type`

In [10]:
using CSV, DataFrames, MySQL

In [10]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql)

db_exec (generic function with 1 method)

In [ ]:
sql = "SELECT id_url_type as id, url_type, url_name_formatter, url_formatter, url_value_example, display_order " *
    "FROM wiag.external_url_type"
df_authority = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [ ]:
df_authority[1:5, [:id, :url_type, :url_name_formatter]]

In [ ]:
Wds.filltable!("authority", df_authority, clear_table=true);

Gemeinsamen Verbundkatalog (GVK) aufnehmen

In [ ]:
sql_gvk = "INSERT INTO authority (id, url_name_formatter, url_formatter, url_value_example)" *
    "VALUES (53, 'GVK - Gemeinsamer Verbundkatalog', 'https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=', '02621539X')"

In [ ]:
DBInterface.execute(Wds.dbwiag, sql_gvk);

*TODO* evtl. `url_type` für GVK ergänzen

Personendatenbank der Germania Sacra aufnehmen

In [11]:
sql = "INSERT INTO authority (id, url_name_formatter, url_formatter, url_value_example)" *
    "VALUES (200, 'Personendatenbank der Germania Sacra', 'http://personendatenbank.germania-sacra.de/index/gsn/', '079-00974-001')"
db_exec(sql);

Externe URLs  
*2022-04-28*

In [10]:
using CSV, DataFrames

In [17]:
file_name = joinpath(wiag2_data_path, "tbl_external_url_type.csv")
df_authority_url = CSV.read(file_name, DataFrame, delim=";");

Addiere zu der ID einen Offset von 1000

In [12]:
names(df_authority_url)

6-element Vector{String}:
 "id_url_type"
 "url_type"
 "url_name_formatter"
 "url_formatter"
 "url_value_example"
 "display_order"

In [18]:
id_offset = 1000;
transform!(df_authority_url, :id_url_type => ByRow(x -> x + id_offset) => :id);

In [20]:
df_authority_url[11:15, [:id, :id_url_type, :url_type, :url_name_formatter]]

,id,id_url_type,url_type,url_name_formatter
,Int64,Int64,String31,String
1,1026,26,Referenzdatenbanken,"Datenbank der Kulturgüter in der Region Trier, ID"
2,1027,27,Referenzdatenbanken,"Digitale Westfälische Urkunden-Datenbank (DWUD), ID"
3,1029,29,Referenzdatenbanken,"Hereditas Monasteriorum Knowledge Portal, Poland"
4,1031,31,Referenzdatenbanken,"Historisches Lexikon Bayerns, Artikel"
5,1036,36,Externe Online-Ressourcen,Österreichisches Musiklexikon Online


In [21]:
table_name = "authority"
Wds.filltable!(table_name, select(df_authority_url, Not(:id_url_type)), clear_table = false);

┌ Info: Rows inserted: 41
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


## Quellen/Referenzen

### Referenzen für Bischöfe und Bistümer
Quelle: Tabelle `wiag.reference`

In [ ]:
sql = "SELECT id_ref as id, " *
    "full_citation, " *
    "author_editor, " *
    "onlineressource as online_resource, " *
    "short_title as title_short, " *
    "ri_opac_id, " *
    "year_of_publication as year_publication, " *
    "isbn, " *
    "sort as display_order " *
    "FROM wiag.reference " *
    "WHERE id_ref <= 5"
df_ref_gatz = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [ ]:
df_ref_gatz[:, [:id, :title_short, :isbn]]

In [ ]:
Wds.filltable!("reference_volume", df_ref_gatz)

GVK in `ref_id_external` eintragen

In [10]:
sql = "SELECT id_ref as reference_id, " *
    "53 as authority_id, " *
    "short_title as title_short, " *
    "gbv " *    
    "FROM wiag.reference " *
    "WHERE id_ref <= 5"
df_ref_ext = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [11]:
filter(:gbv => !ismissing, df_ref_ext)

,reference_id,authority_id,title_short,gbv
,Int32,Int64,String?,String?
1,1,53,"Gatz, Bischöfe 1198 bis 1448",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=336676557
2,2,53,"Gatz, Bistümer",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=365121304
3,3,53,"Gatz, Bischöfe 1448 bis 1648",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=215267079
4,4,53,"Gatz, Bischöfe 1648 bis 1803",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=02621539X
5,5,53,"Gatz, Bischöfe 1785/1803 bis 1945",https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=024331090


In [13]:
function extract_ppn(s)
    rgm = match(r"PPN=(.+$)", s)
    ppn = nothing
    if !isnothing(rgm)
        ppn = rgm[1]
    end
end

extract_ppn (generic function with 1 method)

In [ ]:
extract_ppn("https://kxp.k10plus.de/DB=2.1/PPNSET?PPN=215267079")

In [14]:
df_ref_ext[!, :value] .= extract_ppn.(df_ref_ext[!, :gbv])

5-element Vector{SubString{String}}:
 "336676557"
 "365121304"
 "215267079"
 "02621539X"
 "024331090"

In [ ]:
Wds.filltable!("ref_id_external", df_ref_ext[!, [:reference_id, :authority_id, :value]])

### Referenzen für Domherren der Germania Sacra
Quelle: Tabelle `gso_in_202202.books`

In [30]:
using DataFrames

In [20]:
gso_db = "gso_in_202202"
table_name = "books"
sql = "SELECT id, nummer, titel, kurztitel, autoren, jahr, `order` as gs_display_order, uri
FROM $(gso_db).$(table_name)"
df_ref_gs = Wds.sql_df(sql);

In [11]:
size(df_ref_gs, 1)

87

In [14]:
df_ref_gs[5:10, [:id, :nummer, :kurztitel, :gs_display_order]]

,id,nummer,kurztitel,gs_display_order
,Int32,String?,String,Int32?
1,6,"AF II, 1937","AF II, Bistum Bamberg",6
2,7,"AF II, 1966","AF II, Bistum Bamberg, Pfarreiorganisation",7
3,8,"AF III, 1938","AF III, Archidiakonat Xanten",8
4,9,NF 1,"NF 1, Würzburger Bischöfe bis 1254",9
5,10,NF 2,"NF 2, Cistercienserabtei Altenberg",10
6,11,NF 3,"NF 3, Schwesternhäuser Augustinerregel, Münster",11


In [15]:
item_type_canon_gs = 6

6

In [21]:
df_ref_gs[!, :item_type_id] .= item_type_canon_gs;

In [32]:
columns = [
    :titel => :full_citation,
    :kurztitel => :title_short,
    :nummer => :gs_volume_nr,
    :autoren => :author_editor,
    :jahr => :year_publication,
    :gs_display_order => :display_order,
    :uri => :gs_url,
    :id => :reference_id,
    :item_type_id => :item_type_id
]

9-element Vector{Pair{Symbol, Symbol}}:
            :titel => :full_citation
        :kurztitel => :title_short
           :nummer => :gs_volume_nr
          :autoren => :author_editor
             :jahr => :year_publication
 :gs_display_order => :display_order
              :uri => :gs_url
               :id => :reference_id
     :item_type_id => :item_type_id

In [27]:
names(df_ref_gs)

9-element Vector{String}:
 "id"
 "nummer"
 "titel"
 "kurztitel"
 "autoren"
 "jahr"
 "gs_display_order"
 "uri"
 "item_type_id"

In [33]:
table_name = "reference_volume"
Wds.filltable!(table_name, select(df_ref_gs, columns))

┌ Info: Rows inserted: 87
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


87

## Ämter
Quelle: Access, `GS_Domherren_DB` - `tbl_aemterliste_normiert`

*2022-01-13*

In [21]:
using MySQL, DataFrames, Dates, CSV

In [19]:
item_type_id = 8;

In [16]:
# Quelle CSV
filename = joinpath(wiag2_data_path, "tbl_Aemterliste_normiert.csv")
df_role = CSV.read(filename, DataFrame);

In [26]:
# Quelle Datenbank
sql = "SELECT id_amt_norm AS id_in_source, AmtNorm AS name, Bemerkung AS note 
FROM domherr.tbl_aemterliste_normiert"
df_role = Wds.sql_df(sql);

In [17]:
Wds.clean_up!(df_role);

Einträge in `item`

Bestehende Einträge löschen

In [22]:
table_name = "item";
sql = "DELETE FROM $(table_name) WHERE item_type_id = $(item_type_id)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM item WHERE item_type_id = 8", 0, -1, 153, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [27]:
names(df_role)

11-element Vector{String}:
 "id_amt_norm"
 "AmtNorm"
 "Bemerkung"
 "GSRegID"
 "Synonym"
 "Oberbegriff"
 "Plural"
 "GND"
 "note_intern"
 "definition"
 "domherren_db"

In [28]:
columns = [
    :id_amt_norm => :id_in_source,
    :AmtNorm => :item_in_source,
]

2-element Vector{Pair{Symbol, Symbol}}:
 :id_amt_norm => :id_in_source
     :AmtNorm => :item_in_source

In [33]:
df_item = select(df_role, columns);

In [30]:
user_wiag_id = 1;
time_stamp = Dates.format(now(), dateformat"yyyy-mm-dd HH:MM");

In [34]:
insertcols!(
    df_item, 
    3,
    :item_type_id => item_type_id,
    :edit_status => "importiert",
    :created_by => user_wiag_id,
    :date_created => time_stamp,
    :changed_by => user_wiag_id,
    :date_changed => time_stamp,
    :is_online => 1
);

In [35]:
table_name = "item";
Wds.filltable!(table_name, df_item, clear_table = false)

┌ Info: Rows inserted: 153
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


153

IDs auslesen

In [36]:
table_name = "item"
sql = "SELECT id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_id)"
dfx = Wds.sql_df(sql);

In [39]:
transform!(df_role, :id_amt_norm => ByRow(string) => :id_amt_norm);
df_role_db = innerjoin(dfx, df_role, on = :id_in_source => :id_amt_norm);

In [43]:
names(df_role_db)

12-element Vector{String}:
 "id"
 "id_in_source"
 "AmtNorm"
 "Bemerkung"
 "GSRegID"
 "Synonym"
 "Oberbegriff"
 "Plural"
 "GND"
 "note_intern"
 "definition"
 "domherren_db"

In [44]:
columns = [
    :id => :id, 
    :Bemerkung => :note, 
    :AmtNorm => :name]

3-element Vector{Pair{Symbol, Symbol}}:
        :id => :id
 :Bemerkung => :note
   :AmtNorm => :name

In [45]:
table_name = "role"
Wds.filltable!(table_name, select(df_role_db, columns), clear_table = true)

┌ Info: Rows inserted: 153
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


153